In [97]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import Pipeline
from scipy import sparse
from sklearn.svm import LinearSVC
import json
from sklearn.model_selection import train_test_split
import copy
import requests
import time

# LET'S START

In [132]:
train = pd.read_json('train.json', orient = 'columns', encoding='utf-8')
print(train)

         id sentiment                                               text
0      1945  negative  Досудебное расследование по факту покупки ЕНПФ...
1      1957  negative  Медики рассказали о состоянии пострадавшего му...
2      1969  negative  Прошел почти год, как железнодорожным оператор...
3      1973  negative  По итогам 12 месяцев 2016 года на территории р...
4      1975  negative  Астана. 21 ноября. Kazakhstan Today - Агентств...
5      1980  negative  Об аресте бывшего министра национальной эконом...
6      1982  negative  Двое налетчиков совершили нападение на охранни...
7      1983  negative  "Самрук-Энерго" в I квартале почти вдвое снизи...
8      1985  negative  В 2016 году почти половина коррупционных прест...
9      1988  negative  Google+\nЧт 19 янв 2017 12:35:02\nОдин из фигу...
10     2012  negative  \nАлматы. 20 октября. КазТАГ – S&P Global Rati...
11     2020  negative  \nДолгожданный план, названный Министерством н...
12     2032  negative  /KASE, 29.12.16/ – АО "KEGOC

In [133]:
stop = set(stopwords.words("russian"))

In [134]:
def review_words(raw_review):
    review_txt = BeautifulSoup(raw_review, "lxml")
    letter_only = re.sub(r'[^\w]',' ',review_txt.get_text())
    words = letter_only.lower().split()
    meaningfl_words = [w for w in words if not w in stop]
    return(" ".join(meaningfl_words))


In [135]:
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
num_reviews = train.text.size
clean_reviews = []

for i in range(0,num_reviews):
    clean_reviews.append(review_words(train.text[i]))
vectorizer = CountVectorizer(analyzer='word',tokenizer=None,preprocessor = None, stop_words = None, max_features = 500)
train_data_features= vectorizer.fit_transform(clean_reviews)
train_data_features = train_data_features.toarray()
vocab = vectorizer.get_feature_names()

dist = np.sum(train_data_features,axis=0)

In [142]:
print(clean_reviews)

['ребятки будем очень очень благодарны сможет одолжить android телефон четверга тестинга одного проекта будем аккуратными бесплатно', 'посоветуйте аналоги сайтов baskino or gidonline смотреть оригинальной озвучке', 'ребят очень нужна ваша помощь пейпер горит сёрви рисерча займет 15 секунд буквально https goo gl forms 8s3ppemokfb7cd5u2', 'библиотека завтра работает', 'public opinion можете плз подсказать какого ридинга первый вопрос неделя плзззззз шоколадку', 'кого капельку супер клея взять очки сломала', 'чекаут делают последний уходит', 'парни охраняете своих девушек клубах', 'какие йоги дает файналы калк 3', 'вижу университете красивых интересных девушек знаю завести ними контакт подойти прямо живую найти социальных сетях вк инстаграм начать общаться какие лайфхаки советы', 'те брали генкем2 старые файналы мидки куизы лайкните пожалуйстааа', 'завтра профы будут универе выходные', 'нравится сэндвичи сикс инч многим моим друзьям вкусные это единственное спасает желудок ночью особенно 

In [143]:
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [144]:
import codecs
atok='601a211e49b2cdb3b7f9653bea9bf9f1bbbb012264f0f9f783505a819d92c976d97eac877ced521fd60d8';
print('OK')
r = requests.get('https://api.vk.com/method/wall.get',
                     params ={'owner_id': '-58689213',
                              'count': 1,
                              'offset':0,
                              'access_token':atok,
                              'v': 5.92})
wall = r.json()
number_of_posts = 100#wall['response']['count'];
print(number_of_posts)
offset = 0;
all_posts =[];
while True:
    time.sleep(1)
    r = requests.get('https://api.vk.com/method/wall.get',
                     params ={'owner_id': '-58689213',
                              'count': 100, 'offset': offset,
                              'offset': 0,
                              'access_token':atok,
                              'v': 5.92})
    wall = r.json()
    post = wall['response']['items'];
    all_posts.extend(post)
    if len(all_posts) >= number_of_posts:
        break;
    else:
        offset = offset + 100


OK
100


In [145]:
ttest = [pst['text'] for pst in all_posts]
print(ttest[0])
print(all_posts)


ребятки, будем очень-очень благодарны, если кто-нибудь сможет одолжить Android телефон до четверга для тестинга одного проекта. будем аккуратными. не за бесплатно :)
[{'id': 815688, 'from_id': 209000128, 'owner_id': -58689213, 'date': 1543757624, 'marked_as_ads': 0, 'post_type': 'post', 'text': 'ребятки, будем очень-очень благодарны, если кто-нибудь сможет одолжить Android телефон до четверга для тестинга одного проекта. будем аккуратными. не за бесплатно :)', 'post_source': {'type': 'vk'}, 'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True}, 'likes': {'count': 0, 'user_likes': 0, 'can_like': 1, 'can_publish': 0}, 'reposts': {'count': 0, 'user_reposted': 0}, 'is_favorite': False}, {'id': 815686, 'from_id': 321163410, 'owner_id': -58689213, 'date': 1543757431, 'marked_as_ads': 0, 'post_type': 'post', 'text': 'посоветуйте аналоги сайтов baskino or gidonline, чтобы можно было смотреть в оригинальной озвучке', 'post_source': {'type': 'vk'}, 'comments': {'count': 1, 'can_post':

In [146]:
xgb = RandomForestClassifier(n_estimators = 200)
xgb.fit( train_data_features, train.sentiment)
from sklearn.naive_bayes import GaussianNB

In [147]:
num_reviews = len(ttest)

clean_reviews = []
for i in range(0,num_reviews):
    clean_reviews.append(review_words(ttest[i]))
    
test_features = vectorizer.transform(clean_reviews)
test_features = test_features.toarray() 


In [148]:
result = xgb.predict(test_features)
output = pd.DataFrame(data = {"sentiment":result})
output.to_csv("output.csv",index=False, quoting=3)
print("yeeeeah")

yeeeeah
